# T<sub>2</sub> Ramsey Characterization

This experiment serves as one of the series of experiments used to characterize a single qubit. Its purpose is to determine two of the qubit's properties: *Ramsey* or *detuning frequency* and $T_2\ast$. The rough frequency of the qubit was already determined previously. The control pulses are based on this frequency. 

In this experiment, we would like to get a more precise estimate of the qubit's frequency. The difference between the frequency used for the control rotation pulses, and the precise frequency is called the *detuning frequency*. This part of the experiment is called a *Ramsey Experiment*. $T_2\ast$ represents the rate of decay toward a mixed state, when the qubit is initialized to the |+⟩ state.

Since the detuning frequency is relatively small, we add a phase gate to the circuit to enable better measurement. The actual frequency measured is the sum of the detuning frequency and the user induced *oscillation frequency* (osc_freq parameter).

In [1]:
import qiskit
from qiskit_experiments.library import T2Ramsey

The circuit used for the experiment comprises the following:

    1. Hadamard gate
    2. delay
    3. RZ gate that rotates the qubit in the x-y plane 
    4. Hadamard gate
    5. measurement

During the delay time, we expect the qubit to precess about the z-axis. If the p gate and the precession offset each other perfectly, then the qubit will arrive at the |0⟩ state (after the second Hadamard gate). By varying the extension of the delays, we get a series of oscillations of the qubit state between the |0⟩ and |1⟩ states. We can draw the graph of the resulting function, and can analytically extract the desired values.

In [2]:
# set the computation units to microseconds
unit = "us"  # microseconds
qubit = 0
# set the desired delays
delays = list(range(1, 50, 1))

In [3]:
# Create a T2Ramsey experiment. Print the first circuit as an example
exp1 = T2Ramsey(qubit, delays, unit=unit, osc_freq=0.1)
print(exp1.circuits()[0])

     ┌───┐┌──────────────┐┌─────────┐ ░ ┌───┐ ░ ┌─┐
q_0: ┤ H ├┤ Delay(1[us]) ├┤ Rz(π/5) ├─░─┤ H ├─░─┤M├
     └───┘└──────────────┘└─────────┘ ░ └───┘ ░ └╥┘
c: 1/════════════════════════════════════════════╩═
                                                 0 


We run the experiment on a simple, simulated backend, created specifically for this experiment's tutorial.

In [4]:
from qiskit_experiments.test.t2ramsey_backend import T2RamseyBackend
# FakeJob is a wrapper for the backend, to give it the form of a job
from qiskit_experiments.test.utils import FakeJob
import qiskit_experiments.matplotlib
from qiskit_experiments.matplotlib import pyplot, requires_matplotlib
from qiskit_experiments.matplotlib import HAS_MATPLOTLIB

conversion_factor = 1e-6
# The behavior of the backend is determined by the following parameters
backend = T2RamseyBackend(
    p0={
        "a": [0.5],
        "t2ramsey": [20.0],
        "f": [0.11],
        "phi": [0.0],
        "b": [0.5],
    },
    initial_prob_plus=[0.0],
    readout0to1=[0.02],
    readout1to0=[0.02],
    conversion_factor=conversion_factor,
)

ImportError: cannot import name 'HAS_MATPLOTLIB' from 'qiskit_experiments.matplotlib' (/home/merav/work/osc_freq/qiskit_experiments/matplotlib.py)

The resulting graph will have the form:
$f(t) = a^{-t/T_2*} \cdot \cos(2 \pi f t + \phi) + b$
where *t* is the delay, $T_2*$ is the decay factor, and *f* is the detuning frequency.
`conversion_factor` is a scaling factor that depends on the measurement units used. It is 1E-6 here, because the unit is microseconds.

In [ ]:
exp1.set_analysis_options(user_p0=None, plot=True)
expdata1 = exp1.run(backend=backend, shots=2000)
expdata1.block_for_results()  # Wait for job/analysis to finish.

# Display the figure
display(expdata1.figure(0))

In [ ]:
# Print results
for result in expdata1.analysis_results():
    print(result)

Additional fitter result data is stored in the `result.extra` field

In [ ]:
expdata1.analysis_results("T2").extra

### Providing initial user estimates
The user can provide initial estimates for the parameters to help the analysis process. Because the curve is expected to decay toward $0.5$, the natural choice for parameters $A$ and $B$ is $0.5$. Varying the value of $\phi$ will shift the graph along the x-axis. Since this is not of interest to us, we can safely initialize $\phi$ to 0. In this experiment, `t2ramsey` and `f` are the parameters of interest. Good estimates for them are values computed in previous experiments on this qubit or a similar values computed for other qubits.

In [ ]:
from qiskit_experiments.library.characterization import T2RamseyAnalysis
user_p0={
    "A": 0.5,
    "t2ramsey": 20.0,
    "f": 0.11,
    "phi": 0,
    "B": 0.5
        }
exp_with_p0 = T2Ramsey(qubit, delays, unit=unit, osc_freq=0.1)
exp_with_p0.set_analysis_options(user_p0=user_p0, plot=True)
expdata_with_p0 = exp_with_p0.run(backend=backend, shots=2000)
expdata_with_p0.block_for_results()

# Display fit figure
display(expdata_with_p0.figure(0))

In [ ]:
# Print results
for result in expdata_with_p0.analysis_results():
    print(result)

The units can be changed, but the output in the result is always given in seconds. The units in the backend must be adjusted accordingly.

In [ ]:
from qiskit.utils import apply_prefix

unit = "ns"
delays = list(range(1000, 50000, 1000))
conversion_factor = apply_prefix(1, unit)
print(conversion_factor)

In [ ]:
p0 = {
    "a": [0.5],
    "t2ramsey": [20000],
    "f": [0.0001],
    "phi": [0.0],
    "b": [0.5],
}
backend_in_ns = T2RamseyBackend(
    p0=p0,
    initial_prob_plus=[0.0],
    readout0to1=[0.02],
    readout1to0=[0.02],
    conversion_factor=conversion_factor,
)
exp_in_ns = T2Ramsey(qubit, delays, unit=unit, osc_freq=0.0001)
user_p0_ns = {
    "A": 0.5,
    "t2ramsey": 20000.0,
    "f": 0.00011,
    "phi": 0,
    "B": 0.5
        }
exp_in_ns.set_analysis_options(user_p0=user_p0_ns, plot=True)

# Run experiment
expdata_in_ns = exp_in_ns.run(backend=backend_in_ns, shots=2000).block_for_results()

# Display Figure
display(expdata_in_ns.figure(0))

In [ ]:
# Print Results
for result in expdata_in_ns.analysis_results():
    print(result)

In [ ]:
import qiskit.tools.jupyter
%qiskit_copyright